![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### MySQL and MariaDB for Python Developers
# Fine tuning table data definitions

In this project, we will use the MySQL data definition language to optimize the airline tweets that we moved into MySQL in an earlier project.

You will need access to a MySQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See ` https://hub.docker.com/_/mysql` for details on that option.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Refactoring a database (enumerations)**

The table describing airline tweets was created in a prior project.  Your solution may not be identical to the sample solution, but most of these issues are likely to apply. A solution is available in the file `make_psql.py` in this projects.

A number of columns were probably created as CHAR or TEXT fields, but have only a small number of distinct values.  Those would be more efficiently and descriptively stored as enumerations.  You can create one within a particular database using code similar to (example from MySQL documentation):

```sql
CREATE TABLE shirts (
    name VARCHAR(40),
    size ENUM('x-small', 'small', 'medium', 'large', 'x-large')
);
```

Such an enumeration would be useful, for example, for `airline_sentiment`:

```sql
ine=# SELECT DISTINCT airline_sentiment FROM Tweets;
 airline_sentiment
-------------------
 negative
 positive
 neutral
(3 rows)
```

Create enumerated types for every column where it is relevant, and update the table `Tweets` appropriately.  You may want to create a copy of the `Tweets` table to work with instead.

**Solution**

As a first step, we should find out all columns that have a small number of distinct values.

In [1]:
import mysql.connector
user, pwd = 'ine_student', 'ine-password'
host, port = 'localhost', '3306'
conn = mysql.connector.connect(database='ine', host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

In [2]:
def table_schema(table_name):
    import pandas as pd
    cur.execute(f"SHOW columns FROM {table_name}")
    info_cols = [c[0] for c in cur.description]
    schema = cur.fetchall()
    df = pd.DataFrame(schema, columns=info_cols)
    # Cleaner to show DataFrame with str rather than bytes
    df['Type'] = df.Type.str.decode('utf-8')
    # And nullable as Bool value
    df['Null'] = df.Null == 'YES'
    return df

In [3]:
cur.execute("DROP TABLE IF EXISTS Tweets_new;")
cur.execute("CREATE TABLE Tweets_new SELECT * FROM Tweets;")
conn.commit()

In [4]:
schema = table_schema('Tweets_new')
schema

,Field,Type,Null,Key,Default,Extra
0,tweet_id,"decimal(18,0)",False,,None,
1,airline_sentiment,text,True,,None,
2,airline_sentiment_confidence,double,True,,None,
3,negativereason,text,True,,None,
4,negativereason_confidence,double,True,,None,
5,airline,text,True,,None,
6,airline_sentiment_gold,text,True,,None,
7,name,text,True,,None,
8,negativereason_gold,text,True,,None,
9,retweet_count,int,True,,None,


In [5]:
categories = dict()
for col in schema.Field:
    cur.execute(f"SELECT count(DISTINCT {col}) FROM Tweets_new;")
    num = cur.fetchone()[0]
    print(f"{col} has {num} distinct values")
    if num < 10:
        cur.execute(f"SELECT DISTINCT {col} FROM Tweets_new;")
        categories[col] = [t[0] for t in cur]

tweet_id has 10416 distinct values
airline_sentiment has 3 distinct values
airline_sentiment_confidence has 966 distinct values
negativereason has 11 distinct values
negativereason_confidence has 1403 distinct values
airline has 6 distinct values
airline_sentiment_gold has 4 distinct values
name has 5621 distinct values
negativereason_gold has 14 distinct values
retweet_count has 13 distinct values
text has 10413 distinct values
tweet_coord has 614 distinct values
tweet_created has 10294 distinct values
tweet_location has 2120 distinct values
user_timezone has 74 distinct values


My initial conversion had an empty string for sentiment, which feels like a data problem. I tranformed it to a NULL.  Your initial conversion may not have had this problem.

In [6]:
cur.execute("UPDATE Tweets_new SET airline_sentiment_gold = NULL WHERE airline_sentiment_gold = '';")
cur.execute("SELECT DISTINCT airline_sentiment_gold FROM Tweets_new;")
cur.fetchall()

[(None,), ('neutral',), ('negative',), ('positive',)]

In [7]:
sql_enum_sentiment = """
ALTER TABLE Tweets_new
MODIFY COLUMN airline_sentiment ENUM ('negative', 'positive', 'neutral');
"""
cur.execute(sql_enum_sentiment)

In [8]:
sql_enum_sentiment_gold = """
ALTER TABLE Tweets_new
MODIFY COLUMN airline_sentiment_gold ENUM ('negative', 'positive', 'neutral');
"""
cur.execute(sql_enum_sentiment_gold)

In [9]:
sql_airlines = """
ALTER TABLE Tweets_new
MODIFY COLUMN airline 
  ENUM ('Virgin America', 'Southwest',  'Delta', 'American',  'US Airways', 'United');
"""
cur.execute(sql_airlines)

In [10]:
table_schema('Tweets_new')

,Field,Type,Null,Key,Default,Extra
0,tweet_id,"decimal(18,0)",False,,None,
1,airline_sentiment,"enum('negative','positive','neutral')",True,,None,
2,airline_sentiment_confidence,double,True,,None,
3,negativereason,text,True,,None,
4,negativereason_confidence,double,True,,None,
5,airline,"enum('Virgin America','Southwest','Delta','Ame...",True,,None,
6,airline_sentiment_gold,"enum('negative','positive','neutral')",True,,None,
7,name,text,True,,None,
8,negativereason_gold,text,True,,None,
9,retweet_count,int,True,,None,


In [11]:
conn.commit()

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Adding foreign keys**

The columns `user_timezone` and `airline` feel like entities about which we would like to be able to store additional information.  For example, what is the UTC offset and daylight savings change schedule at a given timezone?  What is the contact information for a given airline, and what airports do they service?

The this task you need to complete two parts.  First you should create tables `timezones` and `airlines`, and populate them with the values currently existing for the foreign key column.  In each case, simply add a second text column called `description` (although more specific fields would clearly be needed for a production use).

Second, create a foreign key constraint between the `Tweets` table and these new tables that provide information about the entitities. You may want to create a copy of the `Tweets` table to work with instead.

**Solution**

In [12]:
cur.execute("DROP TABLE IF EXISTS Tweets_new")
cur.execute("CREATE TABLE Tweets_new SELECT * FROM Tweets;")
conn.commit()

In [13]:
sql_airlines = [
  "ALTER TABLE Tweets_new MODIFY COLUMN airline CHAR(50)",
  "ALTER TABLE Tweets_new MODIFY COLUMN user_timezone CHAR(50)"
]
for sql in sql_airlines:
    cur.execute(sql)

In [14]:
sql_airlines = """
CREATE TABLE airlines (
    airline CHAR(50) PRIMARY KEY,  -- alternate compact key definition
    description TEXT DEFAULT NULL
    );
"""
cur.execute('DROP TABLE IF EXISTS airlines')
cur.execute(sql_airlines)

In [15]:
sql_timezones = """
CREATE TABLE timezones (
    timezone CHAR(50) PRIMARY KEY,
    description TEXT DEFAULT NULL
    );
"""
cur.execute('DROP TABLE IF EXISTS timezones')
cur.execute(sql_timezones)

In [16]:
sql_populate_airlines = """
INSERT INTO airlines (airline)
SELECT DISTINCT airline FROM Tweets_new;
"""
cur.execute(sql_populate_airlines)

In [17]:
sql_populate_timezones = """
INSERT INTO timezones (timezone)
SELECT DISTINCT user_timezone FROM Tweets_new;
"""
cur.execute(sql_populate_timezones)
conn.commit()

Now we need to add the foreign key.

In [18]:
sql_airline_constraint = """
ALTER TABLE Tweets_new
ADD FOREIGN KEY (airline) 
REFERENCES airlines(airline)
ON DELETE CASCADE;
"""
cur.execute(sql_airline_constraint)
conn.commit()

In [19]:
# After alteration need to close cursor and get new one
cur.close()
cur = conn.cursor()

In [20]:
sql_timezone_constraint = """
ALTER TABLE Tweets_new
ADD FOREIGN KEY (user_timezone)
REFERENCES timezones(timezone)
ON DELETE SET NULL;
"""
cur.execute(sql_timezone_constraint)
conn.commit()

In [21]:
import pandas as pd
cur.execute("SHOW INDEXES FROM Tweets_new;")
pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description]).T

,0,1
Table,Tweets_new,Tweets_new
Non_unique,1,1
Key_name,airline,user_timezone
Seq_in_index,1,1
Column_name,airline,user_timezone
Collation,A,A
Cardinality,0,0
Sub_part,None,None
Packed,None,None
Null,YES,YES


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)